In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.gen_doc import nbdoc
from fastai.gen_doc.nbdoc import *

In [3]:
from pathlib import Path
import inspect
import re
import os

In [4]:
def get_test_dir(elt):
    fp = inspect.getfile(elt)
    fp.index('fastai/fastai')
    test_dir = Path(re.sub(r"fastai/fastai/.*", "fastai/tests", fp))
    if not test_dir.exists(): raise Exception('Could not find test path at this location:', test_dir)
    return test_dir

In [5]:
def submodule(elt):
    modules = elt.__module__.split('.')
    if len(modules) > 2:
        return modules[1]
    return None

In [6]:
def get_test_files(elt):
    test_dir = get_test_dir(elt)
    fp = inspect.getfile(elt)
    def is_match(file_name):
        sub_dir = submodule(elt)
        if sub_dir is not None:
            return re.match(f'test_{sub_dir}\w*{Path(fp).stem}\w*\.py', file_name)
        return re.match(f'test\w*{Path(fp).stem}\w*\.py', file_name)
    matches = [test_dir/o.name for o in os.scandir(test_dir) if is_match(o.name)]
    if len(matches) != 1: 
        print('Could not find exact file match:', matches)
    return matches

In [7]:
def find_direct_test_function_match(elt, lines):
    fn_name = nbdoc.fn_name(elt)
    result = []
    for idx,line in enumerate(lines):
        if re.match(f'^def test_\w*{fn_name}\w*\(.*', line):
            result.append((idx,line))
    return result

In [8]:
def get_lines(file):
    with open(file, 'r') as f:
        return f.readlines()

In [9]:
def _fuzzy_line_match(elt, lines):
    fn_name = nbdoc.fn_name(elt)
    result = []
    for idx,line in enumerate(lines):
        if re.match(f'.*[\s\.\(]{fn_name}[\.\(]', line):
            result.append((idx,line))
    return result

In [10]:
def _get_parent_func(lineno, lines):
    for idx,l in enumerate(reversed(lines[:lineno])):
        if re.match(f'^def test', l):
            return (lineno - (idx+1)), l
    return None

In [11]:
def fuzzy_match(elt, lines):
    fuzzy_line_matches = _fuzzy_line_match(elt, lines)
    fuzzy_matches = [_get_parent_func(lineno, lines) for lineno,line in fuzzy_line_matches]
    fuzzy_matches = list(filter(None.__ne__, fuzzy_matches))
    return fuzzy_matches

In [12]:
def find_test_lines(elt, test_file):
    with open(test_file, 'r') as f:
        lines = f.readlines()
    direct_matches = find_direct_test_function_match(elt, lines)
    fuzzy_matches = fuzzy_match(elt, lines)
    return direct_matches, fuzzy_matches

In [ ]:
def relative_test_path(test_file)
    return '/'.join(test_file.parts[-2:])

In [34]:
def pytest_command(line, test_file):
    test_name = re.match(f'^def (test\w*)', line).groups(0)[0]
    test_path = relative_test_path(test_file)
    return f'pytest -sv -k {test_name} {test_path}'

In [35]:
from fastai.basic_data import DataBunch

In [36]:
from fastai.text.data import TextList

In [37]:
elt = TextList

In [ ]:
SOURCE_URL = 'https://github.com/fastai/fastai/blob/master/'

In [ ]:
def source_link(lno, test_file):
    test_path = relative_test_path(test_file)
    return f'{SOURCE_URL}{test_path}#L{lno}'


In [ ]:
def get_links(lno, line, test_file):
    return source_link(lno, test_file), pytest_command(l, test_file)

In [38]:
def get_tests(elt):
    test_dir = get_test_dir(elt)
    test_files = get_test_files(elt)
    all_direct_matches = []
    all_fuzzy_matches = []
    for test_file in test_files:
        direct_matches, fuzzy_matches = find_test_lines(elt, test_file)
        for lno,l in direct_matches: all_direct_matches.append(get_links(lno,l,test_file))
        for lno,l in fuzzy_matches: all_fuzzy_matches.append(get_links(lno,l,test_file))
    return all_direct_matches, all_fuzzy_matches

In [40]:
get_tests(DataBunch)

([(17, 'pytest -sv -k test_DataBunch_Create tests/test_basic_data.py'),
  (30, 'pytest -sv -k test_DataBunch_onebatch tests/test_basic_data.py'),
  (38, 'pytest -sv -k test_DataBunch_oneitem tests/test_basic_data.py'),
  (45, 'pytest -sv -k test_DataBunch_show_batch tests/test_basic_data.py')],
 [(17, 'pytest -sv -k test_DataBunch_Create tests/test_basic_data.py')])

In [58]:
from nbconvert import HTMLExporter
from IPython.core import page
from IPython.core.display import display, Markdown, HTML

In [132]:
def show_doctest(elt, markdown=True):
    fn_name = nbdoc.fn_name(elt)
    md = f'Tests found for `{fn_name}`:'
    duplicates = set()
    dms, fms = get_tests(elt)
    dm_str = ''
    for link,cmd in dms:
        if link in duplicates: continue
        dm_str += '\n * `{cmd}` [\[source\]]({link})'
        duplicates.add(link)
    if dm_str: md += '\n\nDirect Tests:' + dm_str
        
    
    fm_str = ''
    for link,cmd in fms:
        if link in duplicates: continue
        fm_str += f'\n * `{cmd}` [\[source\]]({link})'
        duplicates.add(link)
    if fm_str: md += '\n\nAncillary Tests:' + fm_str
    md += f'\n\nSkeleton test:\n```\n{create_skeleton_test(elt)}\n```'
    if markdown: display(Markdown(md))
    else: return md

* Hello

In [133]:
display(Markdown('hey \n * hello'))

hey 
 * hello

In [134]:
def doctest(elt):
    md = show_doctest(elt, markdown=False)
    output = HTMLExporter().markdown2html(md)
    try:    page.page({'text/html': output})
    except: display(Markdown(md))

In [135]:
show_doctest(elt)

Tests found for `TextList`:

Ancillary Tests:
 * `pytest -sv -k test_from_folder tests/test_text_data.py` [\[source\]](28)
 * `pytest -sv -k test_filter_classes tests/test_text_data.py` [\[source\]](39)
 * `pytest -sv -k test_regression tests/test_text_data.py` [\[source\]](161)

Skeleton test:
```
def test_TextList():
	pass
```

In [119]:
def create_skeleton_test(elt):
    fn_name = nbdoc.fn_name(elt)
    return f'def test_{fn_name}():\n\tpass'

In [95]:
doctest(elt)

In [65]:
test_dir = get_test_dir(elt); test_dir

PosixPath('/Users/andrewshaw/Projects/ML/fastai/tests')

In [66]:
test_files = get_test_files(elt); test_files

[PosixPath('/Users/andrewshaw/Projects/ML/fastai/tests/test_text_data.py')]

In [67]:
lines = get_lines(test_files[0])

In [68]:
fuzzy_match(elt, lines)

[(31, "    data = (TextList.from_folder(path/'temp')\n"),
 (43, "        data = (TextList.from_folder(path/'temp')\n"),
 (164, "    data = (TextList.from_df(df, path, cols='text')\n")]

In [69]:
find_test_lines(elt, test_files[0])

([],
 [(28, 'def test_from_folder():\n'),
  (39, 'def test_filter_classes():\n'),
  (161, 'def test_regression():\n')])

In [6]:
def show_doc(elt, doc_string:bool=True, full_name:str=None, arg_comments:dict=None, title_level=None, alt_doc_string:str='',
             ignore_warn:bool=False, markdown=True):
    "Show documentation for element `elt`. Supported types: class, Callable, and enum."
    arg_comments = ifnone(arg_comments, {})
    anchor_id = full_name or get_anchor(elt)
    elt = getattr(elt, '__func__', elt)
    full_name = full_name or fn_name(elt)
    if inspect.isclass(elt):
        if is_enum(elt.__class__):   name,args = get_enum_doc(elt, full_name)
        else:                        name,args = get_cls_doc(elt, full_name)
    elif isinstance(elt, Callable):  name,args = format_ft_def(elt, full_name)
    else: raise Exception(f'doc definition not supported for {full_name}')
    source_link = get_function_source(elt) if is_fastai_class(elt) else ""
    title_level = ifnone(title_level, 2 if inspect.isclass(elt) else 4)
    doc =  f'<h{title_level} id="{anchor_id}">{name}{source_link}</h{title_level}>'
    doc += f'\n\n> {args}'
    if doc_string and (inspect.getdoc(elt) or arg_comments):
        doc += format_docstring(elt, arg_comments, alt_doc_string, ignore_warn) + ' '
    if markdown: display(Markdown(doc))
    else: return doc

def doc(elt):
    "Show `show_doc` info in preview window along with link to full docs."
    global use_relative_links
    use_relative_links = False
    elt = getattr(elt, '__func__', elt)
    md = show_doc(elt, markdown=False)
    if is_fastai_class(elt):
        md += f'\n\n<a href="{get_fn_link(elt)}" target="_blank" rel="noreferrer noopener">Show in docs</a>'
    output = HTMLExporter().markdown2html(md)
    use_relative_links = True
    page.page({'text/html': output})
    #display(Markdown(md))